# Lesson 3: Adding RAG

**Lesson objective**: Add a document database to a workflow

In this lab, you’ll parse a resume and load it into a vector store, and use the agent to run basic queries against the documents. You’ll use LlamaParse to parse the documents.

<div style="background-color:#fff1d7; padding:15px;"> <b> Note</b>: Make sure to run the notebook cell by cell. Please try to avoid running all cells at once.</div>

## Importing Libraries

In [1]:
from helper import get_openai_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows
import os

You need nested async for this to work, so let's enable it here. It allows you to nest asyncio event loops within each other. 

*Note:* In asynchronous programming, the event loop is like a continuous cycle that manages the execution of code.

In [2]:
import nest_asyncio
nest_asyncio.apply()

You also need two API keys: 
- OpenAI like you used earlier;
- LlamaCloud API key to use LlamaParse to parse the PDFs. In this notebook, you are provided with such a key. For your personal project, you can get a key at cloud.llamaindex.ai for free.

LlamaParse is an advanced document parser that can read PDFs, Word files, Powerpoints, Excel spreadsheets, and extract information out of complicated PDFs into a form LLMs find easy to understand.

In [3]:
llama_cloud_api_key = get_llama_cloud_api_key()
openai_api_key = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access <code>fake_resume.pdf</code>, <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. The resume is inside the data folder.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips and Help"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Performing Retrieval-Augmented Generation (RAG) on a Resume Document

### 1. Parsing the Resume Document 

Let's start by parsing a resume.

<img width="400" src="images/parsing_res.png">

Using LLamaParse, you will transform the resume into a list of Document objects. By default, a Document object stores text along with some other attributes:
- metadata: a dictionary of annotations that can be appended to the text.
- relationships: a dictionary containing relationships to other Documents.
  

You can tell LlamaParse what kind of document it's parsing, so that it will parse the contents more intelligently. In this case, you tell it that it's reading a resume.

In [4]:
from llama_parse import LlamaParse

In [5]:
documents = LlamaParse(
    api_key=llama_cloud_api_key,
    base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
    result_type="markdown",
    content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
).load_data(
    "data/fake_resume.pdf",
)

Started parsing the file under job_id 857a8b52-a980-4bfc-98c0-b38ad8458e81


This gives you a list of Document objects you can feed to a VectorStoreIndex.

In [6]:
print(documents[2].text)

# Projects

# EcoTrack | GitHub

- Built full-stack application for tracking carbon footprint using React, Node.js, and MongoDB
- Implemented machine learning algorithm for providing personalized sustainability recommendations
- Featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023"

# ChatFlow | Demo

- Developed real-time chat application using WebSocket protocol and React
- Implemented end-to-end encryption and message persistence
- Serves 5000+ monthly active users

# Certifications

- AWS Certified Solutions Architect (2023)
- Google Cloud Professional Developer (2022)
- MongoDB Certified Developer (2021)

# Languages

- English (Native)
- Mandarin Chinese (Fluent)
- Spanish (Intermediate)

# Interests

- Open source contribution
- Tech blogging (15K+ Medium followers)
- Hackathon mentoring
- Rock climbing


### 2. Creating a Vector Store Index


<img width="400" src="images/vector_store_index.png">

You'll now feed the Document objects to `VectorStoreIndex`. The `VectorStoreIndex` will use an embedding model to embed the text, i.e. turn it into vectors that you can search. You'll be using an embedding model provided by OpenAI, which is why we needed an OpenAI key.

The `VectorStoreIndex` will return an index object, which is a data structure that allows you to quickly retrieve relevant context for your query. It's the core foundation for RAG use-cases. You can use indexes to build Query Engines and Chat Engines which enables question & answer and chat over your data.


In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex

In [8]:
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small", 
                                api_key= openai_api_key)
)

### 3. Creating a Query Engine with the Index

With an index, you can create a query engine and ask questions. Let's try it out! Asking questions requires an LLM, so let's use OpenAI again.

In [12]:
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(model="gpt-4o-mini")

In [11]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)
response = query_engine.query("What is this person's name and what was their most recent job?")
print(response)

The person's name is Sarah Chen, and their most recent job is Senior Full Stack Developer at TechFlow Solutions.


### 4. Storing the Index to Disk

Indexes can be persisted to disk. This is useful in a notebook that you might run several times! In a production setting, you would probably use a hosted vector store of some kind. Let's save your index to disk.

In [13]:
storage_dir = "./storage"

index.storage_context.persist(persist_dir=storage_dir)

In [14]:
from llama_index.core import StorageContext, load_index_from_storage

You can check if your index has already been stored, and if it has, you can reload an index from disk using the `load_index_from_storage` method, like this:

In [15]:
# Check if the index is stored on disk
if os.path.exists(storage_dir):
    # Load the index from disk
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    restored_index = load_index_from_storage(storage_context)
else:
    print("Index not found on disk.")

In [16]:
response = restored_index.as_query_engine().query("What is this person's name and what was their most recent job?")
print(response)

This person's name is Sarah Chen and their most recent job was as a Senior Full Stack Developer at TechFlow Solutions in San Francisco, CA.


Congratulations! You have performed retrieval augmented generation (RAG) on a resume document. With proper scaling, this technique can work across databases of thousands of documents!

## Making RAG Agentic

With a RAG pipeline in hand, let's turn it into a tool that can be used by an agent to answer questions. This is a stepping-stone towards creating an agentic system that can perform your larger goal.

In [17]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

First, create a regular python function that performs a RAG query. It's important to give this function a descriptive name, to mark its input and output types, and to include a docstring (that's the thing in triple quotes) which describes what it does. The framework will give all this metadata to the LLM, which will use it to decide what a tool does and whether to use it.

In [18]:
def query_resume(q: str) -> str:
    """Answers questions about a specific resume."""
    # we're using the query engine we already created above
    response = query_engine.query(f"This is a question about the specific resume we have in our database: {q}")
    return response.response

The next step is to create the actual tool. There's a utility function, `FunctionTool.from_defaults`, to do this for you.

In [19]:
resume_tool = FunctionTool.from_defaults(fn=query_resume)

Now you can instantiate a `FunctionCallingAgent` using that tool. There are a number of different agent types supported by LlamaIndex; this one is particularly capable and efficient.

You pass it an array of tools (just one in this case), you give it the same LLM we instantiated earlier, and you set Verbose to true so you get a little more info on what your agent is up to.

In [20]:
agent = FunctionCallingAgent.from_tools(
    tools=[resume_tool],
    llm=llm,
    verbose=True
)

Now you can chat to the agent! Let's ask it a quick question about our applicant.

In [21]:
response = agent.chat("How many years of experience does the applicant have?")
print(response)

> Running step 946447b3-fa8e-4edf-a8d5-0b675f01dafb. Step input: How many years of experience does the applicant have?
Added user message to memory: How many years of experience does the applicant have?
=== Calling Function ===
Calling function: query_resume with args: {"q": "How many years of experience does the applicant have?"}
=== Function Output ===
The applicant has over 6 years of experience as a Full Stack Web Developer.
> Running step cfcb3b31-ccef-4711-bd96-b1302b6d9ee6. Step input: None
=== LLM Response ===
The applicant has over 6 years of experience as a Full Stack Web Developer.
The applicant has over 6 years of experience as a Full Stack Web Developer.


You can see the agent getting the question, adding it to its memory, picking a tool, calling it with appropriate arguments, and getting the output back.

## Wrapping the Agentic RAG into a Workflow

You've now got a RAG pipeline and an agent. Let's now create a similar agentic RAG from scratch using a workflow, which you'll extend in later lessons. You won't rely on any of the things you've already created.

Here's the workflow you will create:
<img width="400" src="images/rag_workflow.png">

It consists of two steps:
1. `set_up` which is triggered by `StartEvent` and emits `QueryEvent`: at this step, the RAG system is set up and the query is passed to the second step;
2. `ask_question` which is triggered by `QueryEvent` and emits `StopEvent`: here the response to the query is generated using the RAG query engine.

In [22]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)

In [23]:
class QueryEvent(Event):
    query: str

In [24]:
class RAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    # the first step will be setup
    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> QueryEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        # define an LLM to work with
        self.llm = OpenAI(model="gpt-4o-mini")

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested your documents
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load your documents
            documents = LlamaParse(
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # either way, create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # now fire off a query event to trigger the next step
        return QueryEvent(query=ev.query)

    # the second step will be to ask a question and return a result immediately
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return StopEvent(result=response.response)

You run it like before, giving it a fake resume we created for you.

In [25]:
w = RAGWorkflow(timeout=120, verbose=False)
result = await w.run(
    resume_file="./data/fake_resume.pdf",
    query="Where is the first place the applicant worked?"
)
print(result)

The first place the applicant worked is StartupHub in San Jose, CA.


There's nothing in this workflow you haven't done before, it's just making things neat and encapsulated.

If you're particularly suspicious, you might notice there's a small bug here: if you run this a second time, with a new resume, this code will find the old resume and not bother to parse it. You don't need to fix that now, but think about how you might fix that.

## Workflow Visualization

You can visualize the workflow you just created.

In [26]:
WORKFLOW_FILE = "workflows/rag_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.QueryEvent'>
workflows/rag_workflow.html


## Congratulations!

You've successfully created an agent with RAG tools. In the next lesson, you'll give your agent more complicated tasks.